In [2]:
import os
import numpy as np
from google import protobuf

import matplotlib.pyplot as plt
import urllib, json
import operator
%matplotlib inline

import re
import shutil
import os
from itertools import izip


import utilitites.parse_log

import tabulate

In [14]:
hosts = {0: 'http://localhost:5000',
         1: 'http://localhost:5001'}

data_folder = os.path.join(os.getcwd(), 'data')

try:
    os.mkdir(data_folder)
except OSError as e:
    pass

In [15]:
models = []
for i, addr in hosts.items():
    response = urllib.urlopen(addr+ "/completed_jobs.json")
    data = json.loads(response.read())
    for m in data['models']:
        m['node'] = i
    models += data['models']

models = filter(lambda m: m['status'] != 'Error', models)
models = filter(lambda m: m['name'] not in ['LeNet', 'GoogLeNet'], models)

In [16]:
fetch_experiment_data = False
if fetch_experiment_data:
    # Download all network information
    for m in models:
        net_id = m['id']
        print "Fetching: ",net_id
        host = hosts[m['node']]
        
        net_url = host  + "/files/" + net_id + "/train_val.prototxt"
        net_str = urllib.urlopen(net_url).read()
        solver_url = host  + "/files/" + net_id + "/solver.prototxt"
        solver_str = urllib.urlopen(solver_url).read()
        log_url = host  + "/files/" + net_id + "/caffe_output.log"
        log_str = urllib.urlopen(log_url).read()
        

        net_path = os.path.join(data_folder, '%s_net.prototxt' % net_id)
        open(net_path, 'wb').write(net_str)
        solver_path = os.path.join(data_folder, '%s_solver.prototxt' % net_id)
        open(solver_path, 'wb').write(solver_str)
        log_path = os.path.join(data_folder, '%s_log.log' % net_id)
        open(log_path, 'wb').write(log_str)

In [17]:
def parse_prototxt(prototxt_str):
    layers = re.findall('yer {(.*?)}.la',prototxt_str, flags=re.DOTALL)
    layer_dicts = []
    for l in layers:
        layer_dict = {}
        
        name = re.findall('name: "(.*?)"',l)[0]
        layer_dict['name'] = name    
        layer_type = re.findall('type: "(.*?)"',l)[0]        
        layer_dict['type'] = layer_type
        
        phase = re.findall('phase: (.*?)\n',l)
        if phase:
            layer_dict['phase'] = phase[0]
                
        batch_size = re.findall('batch_size: (.*?)\n',l)
        if batch_size:
            layer_dict['batch_size'] = batch_size[0]        
            
        lr_mult = re.findall('lr_mult: (.*?)\n',l)    
        if lr_mult:
            layer_dict['lr_mult'] = lr_mult
            
        decay_mult = re.findall('decay_mult: (.*?)\n',l)
        if decay_mult:
            layer_dict['decay_mult'] = decay_mult
                    
        layer_dicts.append(layer_dict)

    return layer_dicts


def get_net_info(net_id):        
    net_path = os.path.join(data_folder, '%s_net.prototxt' % net_id)
    network = parse_prototxt(open(net_path).read())

    res = dict()
    if not network:
        return res
        
    batch_size = (map(lambda l: l.get('batch_size',0),
                      filter(lambda l: l.get('phase')=='TRAIN', network)))
    res['batch_size'] = max(batch_size) if batch_size else None
    return res
    



In [18]:
get_net_info('20160704-001701-30e0')

{'batch_size': '64'}

In [47]:
def get_log_info(net_id):        
    log_path = os.path.join(data_folder, '%s_log.log' % net_id)
    log_str = open(log_path).read()
    
    name_iter = re.finditer('net.cpp:153] Setting up (.*)', log_str)
    mem_iter = re.finditer('Memory required for data: (.*)', log_str)
    
    mem = dict()
    for n, m in izip(name_iter, mem_iter):
        mem[n.group(1)] = (int(m.group(1)) * 2**-20)
    return mem 
#    return parse_log.parse_log(log_path)

get_log_info('20160704-001701-30e0')
    

{'accuracy': 214.80835723876953,
 'data': 202.239990234375,
 'drop6': 211.10009765625,
 'drop7': 214.10009765625,
 'fc6': 209.10009765625,
 'fc7': 212.10009765625,
 'fc8_species': 214.336181640625,
 'fc8_species_fc8_species_0_split': 214.808349609375,
 'label': 202.240234375,
 'label_label_0_split': 202.24072265625,
 'loss': 214.80835342407227,
 'pool1': 208.10009765625,
 'relu6': 210.10009765625,
 'relu7': 213.10009765625}

In [10]:
<
  
    
def get_network_info(network):
    network = parse_prototxt(prototxt_str)
    res = dict()
    if not network:
        return res
        
    batch_size = (map(lambda l: l.get('batch_size',0),
                      filter(lambda l: l.get('phase')=='TRAIN', network)))
    res['batch_size'] = max(batch_size) if batch_size else None
    return res
    
    
get_network_info(get_network_prototxt(hosts[0], id))['batch_size']

TypeError: cannot concatenate 'str' and 'builtin_function_or_method' objects

In [357]:
keys = sorted(list(set(reduce(operator.add, map(dict.keys, models)))))
print "\n".join(keys)

accuracy (val) last
accuracy (val) max
accuracy (val) min
dataset_id
elapsed
epoch (train) last
epoch (train) max
epoch (train) min
epoch (val) last
epoch (val) max
epoch (val) min
framework
id
learning_rate (train) last
learning_rate (train) max
learning_rate (train) min
loss (train) last
loss (train) max
loss (train) min
loss (val) last
loss (val) max
loss (val) min
name
node
progress
sparkline
status
status_css
submitted
type


In [183]:
f = open('/Users/mads/Desktop/test.csv','wb')

In [164]:
w = csv.DictWriter(f, keys)
w.writeheader()

map(w.writerow, models);


In [370]:
for m in sorted(models, key=lambda m: m.get('name'),):
    print "%50s" % m['name'], 
    print "%8.2f%%" % (100*m.get('accuracy (val) last',0)),
    print "%5i" % m['progress'],
    print "%s" % m['node'],
    print get_network_info(hosts[m['node']], m['id'])['batch_size'],
    print 

                                   AlexNetTransfer    47.30%   100 1 128
                               a1_m0_s1_f2 Alex1st     5.91%     8 0 64
                               a4_m1_s1_f2 Alex1st     7.66%    46 0 32
                               a4_m1_s1_f2 Alex1st    12.84%   100 0 32
                             a4_m2_s1s2_f2 i:conv1    20.25%   100 0 64
                             a4_m2_s1s2_f2 i:conv1    10.34%   100 0 32
                       a4_m2_s1s2_f2 i:conv2 p:3_4    16.53%   100 0 64
                       a4_m2_s1s2_f2 i:conv2 p:3_4    14.95%   100 0 32
                       a4_m2_s1s2_f2 i:conv3 p:5_6     9.59%   100 0 32
                       a4_m2_s1s2_f2 i:conv3 p:5_6    15.82%   100 0 32
              a4_m2_s1s2_f2 i:conv3 p:5_6 batch<64    10.02%   100 0 32
                       a4_m2_s1s2_f2 i:conv5 p:5_6    18.62%   100 0 32
                       a4_m2_s1s2_f2 i:conv5 p:5_6    10.20%    65 0 32
                         a4_m2_s1s2s4s8_f2 i:conv3    18.07%   

In [153]:
def parse_name(name):
    components = name.split()
    d = dict()
    d['src'] = components[0]
    for c in components[1:]:
        if len(c.split(':')) == 2:
            k, v = c.split(':')
            d[k]=v
    return d
            
parse_name(models[0]['name'])    

{u'i': u'conv3', u'p': u'5_6', 'src': u'a4_m2_s1s2_f2'}

In [98]:

from IPython.display import display, Markdown, Latex

d = [{'i':42, 'k':5},{'i':10, 'v':12}]
s = tabulate.tabulate(d, tablefmt="pipe", headers={})
display(Markdown(s))


|   i |   k |   v |
|----:|----:|----:|
|  42 |   5 |     |
|  10 |     |  12 |

In [102]:

models_f = [m.copy() for m in models]
for m in models_f:
    del m['sparkline']


s = tabulate.tabulate(models_f, tablefmt="pipe", headers={})
display(Markdown(s))

|   node | status   |   epoch (val) last | status_css   |   epoch (train) min |   accuracy (val) last |   epoch (val) min |   epoch (train) max |   elapsed | framework   |   learning_rate (train) max |   accuracy (val) min |   progress | dataset_id           |   loss (val) max | id                   |   epoch (train) last |   epoch (val) max | name                                 |   loss (train) last |   submitted |   loss (train) max |   loss (val) last |   loss (val) min |   accuracy (val) max |   learning_rate (train) last | type   |   learning_rate (train) min |   loss (train) min |
|-------:|:---------|-------------------:|:-------------|--------------------:|----------------------:|------------------:|--------------------:|----------:|:------------|----------------------------:|---------------------:|-----------:|:---------------------|-----------------:|:---------------------|---------------------:|------------------:|:-------------------------------------|--------------------:|------------:|-------------------:|------------------:|-----------------:|---------------------:|-----------------------------:|:-------|----------------------------:|-------------------:|
|      0 | Done     |                 80 | success      |                   0 |            0.0959135  |                 0 |            79.9433  | 16588.2   | caffe       |                       0.01  |          0.000120192 |        100 | 20160703-183952-b2cf |          6.87511 | 20160708-110212-1143 |             79.9433  |                80 | a4_m2_s1s2_f2 i:conv3 p:5_6          |         0.00196767  | 1.46797e+09 |            6.88175 |           6.45885 |          5.29855 |           0.0979567  |                  2.06531e-05 | model  |                 2.06531e-05 |        2.46287e-05 |
|      0 | Done     |                100 | success      |                   0 |            0.411298   |                 0 |            99.9367  |  8368.69  | caffe       |                       0.01  |          0.000961538 |        100 | 20160624-120053-bb9a |          7.1858  | 20160708-003123-0d2e |             99.9367  |               100 | ok) raw Alex l:fc7 c:fc8             |         1.03617     | 1.46793e+09 |            7.80699 |           3.04171 |          3.03845 |           0.414183   |                  5.94354e-05 | model  |                 5.94354e-05 |        0.611328    |
|      0 | Done     |                 80 | success      |                   0 |            0.10024    |                 0 |            79.9433  | 16575.2   | caffe       |                       0.01  |          0.000120192 |        100 | 20160703-183952-b2cf |          6.87511 | 20160706-065616-d8f8 |             79.9433  |                80 | a4_m2_s1s2_f2 i:conv3 p:5_6 batch<64 |         0.02067     | 1.46778e+09 |            6.88175 |           6.3088  |          5.24854 |           0.101322   |                  2.06531e-05 | model  |                 2.06531e-05 |        2.87446e-05 |
|      0 | Done     |                 80 | success      |                   0 |            0.139784   |                 0 |            79.9433  |  9823     | caffe       |                       0.01  |          0.000360577 |        100 | 20160703-232308-5546 |          6.87862 | 20160705-221918-6a5b |             79.9433  |                80 | a4_m3_s1s2s4_f2 i:conv5 p:5_6        |         0.0979416   | 1.46775e+09 |            6.89572 |           6.5157  |          5.01169 |           0.140144   |                  2.06475e-05 | model  |                 2.06475e-05 |        0.000411439 |
|      0 | Done     |                 80 | success      |                   0 |            0.202524   |                 0 |            79.9433  |  9193.43  | caffe       |                       0.01  |          0.000961538 |        100 | 20160703-183952-b2cf |          8.50465 | 20160705-221756-e7b1 |             79.9433  |                80 | a4_m2_s1s2_f2 i:conv1                |         0.00312502  | 1.46775e+09 |            6.89168 |           8.48966 |          4.38468 |           0.203365   |                  2.06475e-05 | model  |                 2.06475e-05 |        0.00042857  |
|      0 | Done     |                 80 | success      |                   0 |            0.165264   |                 0 |            79.9433  | 18473.2   | caffe       |                       0.01  |          0           |        100 | 20160703-183952-b2cf |          6.88173 | 20160705-221737-ca2c |             79.9433  |                80 | a4_m2_s1s2_f2 i:conv2 p:3_4          |         0.00125641  | 1.46775e+09 |            6.88167 |           6.14472 |          4.79586 |           0.165986   |                  2.06475e-05 | model  |                 2.06475e-05 |        5.82855e-05 |
|      0 | Done     |                 80 | success      |                   0 |            0.186178   |                 0 |            79.9433  |  9400.59  | caffe       |                       0.001 |          0.000600962 |        100 | 20160703-183952-b2cf |          6.95648 | 20160705-220738-8a08 |             79.9433  |                80 | a4_m2_s1s2_f2 i:conv5 p:5_6          |         0.00528707  | 1.46775e+09 |            6.98786 |           5.59722 |          4.59859 |           0.18726    |                  2.06531e-06 | model  |                 2.06531e-06 |        0.00381639  |
|      0 | Done     |                 80 | success      |                   0 |            0.0629808  |                 0 |            79.9433  | 16181.7   | caffe       |                       0.01  |          0.000841346 |        100 | 20160703-232308-5546 |          6.87965 | 20160705-215037-3a1c |             79.9433  |                80 | a4_m3_s1s2s4_f2 i:conv3 p:2_3        |         0.000134584 | 1.46775e+09 |            6.87064 |           6.75043 |          5.47281 |           0.0647837  |                  0.000168912 | model  |                 0.000168912 |        4.08556e-05 |
|      0 | Done     |                 80 | success      |                   0 |            0.214762   |                 0 |            79.9362  |  3376.42  | caffe       |                       0.001 |          0.00142857  |        100 | 20160704-151835-5ba8 |          6.97512 | 20160705-214944-ddc0 |             79.9362  |                80 | a4_m2_s1s2s4s8_f2 i:conv5 p:1_2      |         0.857368    | 1.46775e+09 |            7.17786 |           4.70575 |          4.39423 |           0.218095   |                  2.06569e-06 | model  |                 2.06569e-06 |        0.67938     |
|      0 | Aborted  |                 56 | warning      |                   0 |            0.00785714 |                 0 |            56.1844  |   814     | caffe       |                       0.01  |          0.0027381   |         56 | 20160704-151835-5ba8 |          6.89547 | 20160705-130445-443c |             56.1844  |                56 | a4_m2_s1s2s4s8_f2 i:conv5 p:2_3 adam |         6.68809     | 1.46772e+09 |           11.0344  |           6.58626 |          6.58569 |           0.00809524 |                  0.01        | model  |                 0.01        |        6.50862     |
|      0 | Aborted  |                 54 | warning      |                   0 |            0.120952   |                 0 |            54.1135  |  4456.83  | caffe       |                       0.01  |          0.00107143  |         54 | 20160704-151835-5ba8 |          6.90126 | 20160705-071507-f290 |             54.1135  |                54 | a4_m2_s1s2s4s8_f2 i:conv4            |         0.159153    | 1.4677e+09  |            6.91318 |           6.7545  |          5.01313 |           0.120952   |                  0.00109856  | model  |                 0.00109856  |        0.0261466   |
|      0 | Done     |                100 | success      |                   0 |            0.180714   |                 0 |            99.9113  | 25570.5   | caffe       |                       0.01  |          0.000714286 |        100 | 20160704-151835-5ba8 |          6.88059 | 20160704-232348-6577 |             99.9113  |               100 | a4_m2_s1s2s4s8_f2 i:conv3            |         0.00398609  | 1.46767e+09 |            6.87528 |           5.65547 |          4.53846 |           0.182143   |                  0.00016945  | model  |                 0.00016945  |        4.54235e-05 |
|      0 | Done     |                100 | success      |                   0 |            0.242976   |                 0 |            99.9504  |  1406.83  | caffe       |                       0.005 |          0.0027381   |        100 | 20160704-151835-5ba8 |          6.89547 | 20160704-171221-2222 |             99.9504  |               100 | a4_m2_s1s2s4s8_f2 i:conv5 p:2_3      |         0.143595    | 1.46765e+09 |            6.96744 |           5.41082 |          4.27598 |           0.244048   |                  1.0304e-05  | model  |                 1.0304e-05  |        0.0938605   |
|      0 | Done     |                100 | success      |                   0 |            0.0705529  |                 0 |            99.898   | 12287     | caffe       |                       0.01  |          0.00108173  |        100 | 20160704-151835-5ba8 |          9.26267 | 20160704-171110-ef79 |             99.898   |               100 | a4_m3_s1s2s4_f2 Alex4th              |         0.827191    | 1.46765e+09 |            6.99755 |           9.26239 |          5.67566 |           0.0716346  |                  2.06723e-05 | model  |                 2.06723e-05 |        0.00193246  |
|      0 | Done     |                100 | success      |                   0 |            0.211905   |                 0 |            99.9504  |  4026.69  | caffe       |                       0.01  |          0.00119048  |        100 | 20160704-151835-5ba8 |          6.95391 | 20160704-162710-6dc9 |             99.9504  |               100 | a4_m2_s1s2s4s8_f2 i:conv5            |         1.92575     | 1.46764e+09 |            7.13823 |           4.1829  |          4.02057 |           0.213214   |                  2.06081e-05 | model  |                 2.06081e-05 |        1.63758     |
|      0 | Aborted  |                 79 | warning      |                   0 |            0.219762   |                 0 |            79.9362  |  2970.09  | caffe       |                       0.001 |          0.00119048  |         80 | 20160704-151835-5ba8 |          6.95391 | 20160704-154710-e898 |             79.9362  |                79 | a4_m2_s1s2s4s8_f2 i:conv5            |         0.481317    | 1.46764e+09 |            7.13823 |           4.91698 |          4.4165  |           0.222619   |                  7.11014e-06 | model  |                 7.11014e-06 |        0.325209    |
|      0 | Aborted  |                 29 | warning      |                   0 |            0.209524   |                 0 |            29.4184  |  1095.27  | caffe       |                       0.001 |          0.00142857  |         29 | 20160704-151835-5ba8 |          6.97512 | 20160704-152854-7d95 |             29.4184  |                29 | a4_m2_s1s2s4s8_f2 i:conv5 p:1_2      |         0.971091    | 1.46764e+09 |            7.17786 |           4.68772 |          4.3705  |           0.209881   |                  0.000161972 | model  |                 0.000161972 |        0.88843     |
|      0 | Done     |                100 | success      |                   0 |            0.421875   |                 0 |            99.9367  |  8443.84  | caffe       |                       0.01  |          0.00120192  |        100 | 20160624-120053-bb9a |          7.07288 | 20160704-065721-1c42 |             99.9367  |               100 | raw Alex l:conv5 c:conv6             |         0.672011    | 1.46761e+09 |            7.58855 |           2.79306 |          2.77809 |           0.423077   |                  5.94354e-05 | model  |                 5.94354e-05 |        0.364037    |
|      0 | Aborted  |                 34 | warning      |                   0 |            0.041226   |                 0 |            34.9206  |  6953.44  | caffe       |                       0.01  |          0.000841346 |         35 | 20160703-232308-5546 |          6.87965 | 20160704-001911-eb45 |             34.9206  |                34 | a4_m3_s1s2s4_f2 i:conv3 p:2_3        |         0.195313    | 1.46758e+09 |            6.87064 |           6.74296 |          5.47261 |           0.0526442  |                  0.00240597  | model  |                 0.00240597  |        0.195313    |
|      0 | Aborted  |                 53 | warning      |                   0 |            0.140238   |                 0 |            53.9894  |  7175.63  | caffe       |                       0.01  |          0.000357143 |         54 | 20160703-232308-5546 |          6.87864 | 20160704-001743-6578 |             53.9894  |                53 | a4_m3_s1s2s4_f2 i:conv3 p:5_6        |         0.0127431   | 1.46758e+09 |            6.89972 |           6.35072 |          4.7289  |           0.140238   |                  0.00110414  | model  |                 0.00110414  |        0.00165712  |
|      0 | Done     |                100 | success      |                   0 |            0.0850962  |                 0 |            99.898   | 11489.6   | caffe       |                       0.001 |          0.00168269  |        100 | 20160703-232308-5546 |          7.47378 | 20160704-001701-30e0 |             99.898   |               100 | a4_m3_s1s2s4_f2 i:fc6 p:6_7          |         0.201197    | 1.46758e+09 |            9.89976 |           7.01671 |          5.78282 |           0.0858173  |                  1.69382e-05 | model  |                 1.69382e-05 |        0.0827919   |
|      0 | Done     |                100 | success      |                   0 |            0.176548   |                 0 |            99.9113  | 10768.2   | caffe       |                       0.001 |          0.000238095 |        100 | 20160703-232308-5546 |          6.93395 | 20160704-001627-a4de |             99.9113  |               100 | a4_m3_s1s2s4_f2 i:conv5 p:4_5        |         0.175044    | 1.46758e+09 |            7.22499 |           5.34903 |          4.55009 |           0.180833   |                  2.06545e-06 | model  |                 2.06545e-06 |        0.0917473   |
|      0 | Aborted  |                  4 | warning      |                   0 |            0.213582   |                 0 |             4.15385 |   360.139 | caffe       |                       0.01  |          0.00120192  |         14 | 20160624-120053-bb9a |          7.07288 | 20160704-001413-898d |              4.15385 |                 4 | caffenet_trans_cleanFF_lockConv      |         4.57945     | 1.46758e+09 |            7.58855 |           4.0749  |          4.0749  |           0.213582   |                  0.00491532  | model  |                 0.00491532  |        4.19793     |
|      0 | Done     |                 10 | success      |                   0 |            0.13726    |                 0 |             9.97732 |   961.138 | caffe       |                       0.01  |          0.000360577 |        100 | 20160703-232308-5546 |          6.87862 | 20160703-232402-8605 |              9.97732 |                10 | a4_m3_s1s2s4_f2 i:conv5 p:5_6        |         3.77535     | 1.46758e+09 |            6.90218 |           4.52744 |          4.52744 |           0.13726    |                  2.08356e-05 | model  |                 2.08356e-05 |        3.10701     |
|      0 | Aborted  |                  6 | warning      |                   0 |            0.102043   |                 0 |             6.48526 |   621.834 | caffe       |                       0.001 |          0.000600962 |         65 | 20160703-183952-b2cf |          6.95648 | 20160703-211919-c6bb |              6.48526 |                 6 | a4_m2_s1s2_f2 i:conv5 p:5_6          |         4.98111     | 1.46757e+09 |            6.98786 |           4.9674  |          4.9674  |           0.102043   |                  1.80811e-05 | model  |                 1.80811e-05 |        3.67212     |
|      0 | Done     |                 10 | success      |                   0 |            0.158173   |                 0 |             9.97732 |  2054.42  | caffe       |                       0.01  |          0.000120192 |        100 | 20160703-183952-b2cf |          6.87511 | 20160703-195348-6004 |              9.97732 |                10 | a4_m2_s1s2_f2 i:conv3 p:5_6          |         2.63168     | 1.46757e+09 |            6.88175 |           4.43479 |          4.42925 |           0.158173   |                  2.08356e-05 | model  |                 2.08356e-05 |        2.1885      |
|      0 | Done     |                 10 | success      |                   0 |            0.149519   |                 0 |             9.97732 |  2966.25  | caffe       |                       0.01  |          0           |        100 | 20160703-183952-b2cf |          6.88173 | 20160703-195208-cdcf |              9.97732 |                10 | a4_m2_s1s2_f2 i:conv2 p:3_4          |         2.87312     | 1.46757e+09 |            6.88522 |           4.42293 |          4.40505 |           0.149639   |                  2.08356e-05 | model  |                 2.08356e-05 |        2.4489      |
|      0 | Done     |                 10 | success      |                   0 |            0.103365   |                 0 |             9.97732 |   960.117 | caffe       |                       0.01  |          0.000961538 |        100 | 20160703-183952-b2cf |          6.87648 | 20160703-184038-b35f |              9.97732 |                10 | a4_m2_s1s2_f2 i:conv1                |         4.07156     | 1.46756e+09 |            6.89512 |           4.59655 |          4.59655 |           0.103365   |                  2.08356e-05 | model  |                 2.08356e-05 |        3.7671      |
|      0 | Aborted  |                  4 | warning      |                   0 |            0.0765625  |                 0 |             4.61451 |   494.507 | caffe       |                       0.01  |          0           |         46 | 20160703-164652-8de6 |          6.87604 | 20160703-165031-26a6 |              4.61451 |                 4 | a4_m1_s1_f2 Alex1st                  |         4.50514     | 1.46756e+09 |            6.88544 |           4.89094 |          4.89094 |           0.0765625  |                  0.000575371 | model  |                 0.000575371 |        4.38977     |
|      0 | Done     |                 10 | success      |                   0 |            0.128365   |                 0 |             9.97732 |  1031.32  | caffe       |                       0.01  |          0           |        100 | 20160703-164652-8de6 |          6.8761  | 20160703-164719-37a7 |              9.97732 |                10 | a4_m1_s1_f2 Alex1st                  |         4.18931     | 1.46756e+09 |            6.88493 |           4.53678 |          4.53678 |           0.128365   |                  2.08356e-05 | model  |                 2.08356e-05 |        3.67938     |
|      0 | Aborted  |                  7 | warning      |                   0 |            0.0591346  |                 0 |             7.60771 |   492.327 | caffe       |                       0.01  |          0.000480769 |          8 | 20160703-150446-b90d |          6.87491 | 20160703-150520-3997 |              7.60771 |                 7 | a1_m0_s1_f2 Alex1st                  |         4.88616     | 1.46755e+09 |            6.86222 |           5.1691  |          5.1691  |           0.0591346  |                  0.00624521  | model  |                 0.00624521  |        4.60131     |
|      0 | Done     |                100 | success      |                   0 |            0.288452   |                 0 |            99.9113  | 13219.7   | caffe       |                       0.01  |          0.000357143 |        100 | 20160624-120053-bb9a |          6.87745 | 20160625-083048-3086 |             99.9113  |               100 | raw Alex c:conv1 v2                  |         0.113556    | 1.46684e+09 |            6.87752 |           5.60039 |          4.03166 |           0.288929   |                  2.06545e-05 | model  |                 2.06545e-05 |        0.0194873   |
|      0 | Done     |                100 | success      |                   0 |            0.473929   |                 0 |            99.9113  | 13135.5   | caffe       |                       0.01  |          0.000595238 |        100 | 20160624-120053-bb9a |          6.99139 | 20160625-001322-6968 |             99.9113  |               100 | raw Alex t:conv4 c:conv5             |         0.0422956   | 1.46681e+09 |            7.31539 |           3.79226 |          3.06751 |           0.476548   |                  2.06545e-05 | model  |                 2.06545e-05 |        0.0041855   |
|      0 | Done     |                 80 | success      |                   0 |            0.505833   |                 0 |            79.9291  |  7782.42  | caffe       |                       0.01  |          0.00107143  |        100 | 20160624-120053-bb9a |          7.1858  | 20160624-182808-811a |             79.9291  |                80 | raw Alex t:fc7 c:fc8                 |         0.00938635  | 1.46679e+09 |            7.74359 |           3.25371 |          3.01448 |           0.507381   |                  5.94831e-05 | model  |                 5.94831e-05 |        0.00152035  |
|      0 | Done     |                 30 | success      |                   0 |            0.2625     |                 0 |            29.9113  |  3089.05  | caffe       |                       0.01  |          0.00119048  |        100 | 20160624-120053-bb9a |          6.8758  | 20160624-121155-2c79 |             29.9113  |                30 | raw Alex c:conv1                     |         1.53223     | 1.46676e+09 |            6.89773 |           4.01136 |          3.85539 |           0.2625     |                  1e-05       | model  |                 1e-05       |        1.06588     |
|      1 | Done     |                 80 | success      |                   0 |            0.438941   |                 0 |            79.9474  | 10316.9   | caffe       |                       0.01  |          0.00105882  |        100 | 20160704-085910-0cc9 |          7.19445 | 20160708-003847-e5ae |             79.9474  |                80 | raw l:fc7 c:fc8                      |         1.36492     | 1.46793e+09 |            7.7889  |           2.59994 |          2.59225 |           0.440118   |                  5.94101e-05 | model  |                 5.94101e-05 |        1.14179     |
|      1 | Done     |                 80 | success      |                   0 |            0.356429   |                 0 |            79.9362  | 11209.2   | caffe       |                       0.01  |          0.000595238 |        100 | 20160704-085910-0cc9 |          6.88138 | 20160705-214821-0e83 |             79.9362  |                80 | raw l:conv1 c:conv2                  |         0.178203    | 1.46775e+09 |            6.88036 |           4.64287 |          3.6101  |           0.35881    |                  5.94614e-05 | model  |                 5.94614e-05 |        0.0301545   |
|      1 | Done     |                 80 | success      |                   0 |            0.459881   |                 0 |            79.9362  | 10574.5   | caffe       |                       0.01  |          0.000595238 |        100 | 20160704-085910-0cc9 |          6.8887  | 20160705-214810-44a7 |             79.9362  |                80 | raw l:conv2 c:conv3                  |         0.0990544   | 1.46775e+09 |            6.92726 |           3.81492 |          3.00011 |           0.459881   |                  5.94614e-05 | model  |                 5.94614e-05 |        0.0333782   |
|      1 | Done     |                 80 | success      |                   0 |            0.483929   |                 0 |            79.9362  | 10965.6   | caffe       |                       0.01  |          0.000357143 |        100 | 20160704-085910-0cc9 |          6.89603 | 20160705-214758-e0a4 |             79.9362  |                80 | raw l:conv3 c:conv4                  |         0.0749985   | 1.46775e+09 |            7.04033 |           3.46183 |          2.84925 |           0.485119   |                  5.94614e-05 | model  |                 5.94614e-05 |        0.0193339   |
|      1 | Done     |                 80 | success      |                   0 |            0.458809   |                 0 |            79.9362  | 11261     | caffe       |                       0.01  |          0.000595238 |        100 | 20160704-085910-0cc9 |          6.89196 | 20160705-214733-e267 |             79.9362  |                80 | raw t:conv2 c:conv3                  |         0.147325    | 1.46775e+09 |            6.8756  |           3.95081 |          3.06082 |           0.458809   |                  5.94614e-05 | model  |                 5.94614e-05 |        0.0406099   |
|      1 | Done     |                 80 | success      |                   0 |            0.341429   |                 0 |            79.9362  | 10972.1   | caffe       |                       0.01  |          0.000238095 |        100 | 20160704-085910-0cc9 |          6.87963 | 20160705-214721-4d47 |             79.9362  |                80 | raw t:conv1 c:conv2                  |         0.153589    | 1.46775e+09 |            6.88685 |           4.81878 |          3.76925 |           0.343452   |                  5.94614e-05 | model  |                 5.94614e-05 |        0.0539525   |
|      1 | Done     |                 80 | success      |                   0 |            0.485714   |                 0 |            79.9362  | 11002     | caffe       |                       0.01  |          0.00119048  |        100 | 20160704-085910-0cc9 |          6.89734 | 20160705-214703-ac3e |             79.9362  |                80 | raw t:conv3 c:conv4                  |         0.10233     | 1.46775e+09 |            6.98573 |           3.67421 |          2.86813 |           0.485714   |                  5.94614e-05 | model  |                 5.94614e-05 |        0.0172614   |
|      1 | Done     |                100 | success      |                   0 |            0.480714   |                 0 |            99.9504  | 14826.5   | caffe       |                       0.01  |          0.000833333 |        100 | 20160704-085910-0cc9 |          6.96853 | 20160705-131412-ff07 |             99.9504  |               100 | raw l:conv4 c:conv5                  |         0.0488859   | 1.46772e+09 |            7.21973 |           3.40761 |          2.88972 |           0.480714   |                  5.93634e-05 | model  |                 5.93634e-05 |        0.0190695   |
|      1 | Done     |                100 | success      |                   0 |            0.526548   |                 0 |            99.9504  | 14208.6   | caffe       |                       0.01  |          0.000952381 |        100 | 20160704-085910-0cc9 |          7.25231 | 20160705-091349-2744 |             99.9504  |               100 | raw t:fc7 l:conv4 c:fc8              |         0.0366117   | 1.4677e+09  |            7.83147 |           3.06933 |          2.78659 |           0.526667   |                  5.93634e-05 | model  |                 5.93634e-05 |        0.00259915  |
|      1 | Aborted  |                 16 | warning      |                   0 |            0.503452   |                 0 |            16.9014  |  2242.07  | caffe       |                       0.01  |          0.00119048  |         17 | 20160704-085910-0cc9 |          7.20793 | 20160705-075154-d89f |             16.9014  |                16 | raw l:conv4 c:conv5                  |         0.177353    | 1.4677e+09  |            7.7654  |           2.61274 |          2.52025 |           0.503452   |                  0.00420245  | model  |                 0.00420245  |        0.172153    |
|      1 | Aborted  |                 32 | warning      |                   0 |            0.439167   |                 0 |            32.2254  |  4585.85  | caffe       |                       0.01  |          0.00119048  |         32 | 20160704-085910-0cc9 |          7.19983 | 20160705-072003-d85d |             32.2254  |                32 | raw l:fc7 c:fc8                      |         1.37499     | 1.4677e+09  |            7.75781 |           2.60744 |          2.60744 |           0.439167   |                  0.0019149   | model  |                 0.0019149   |        1.14102     |
|      1 | Done     |                100 | success      |                   0 |            0.340714   |                 0 |            99.9437  | 12656.1   | caffe       |                       0.01  |          0.000833333 |        100 | 20160704-085910-0cc9 |          7.06321 | 20160704-232921-df78 |             99.9437  |               100 | caffenet_trans_cleanFF_lockConv_new  |         3.30928     | 1.46767e+09 |            7.59687 |           3.48987 |          3.48413 |           0.342381   |                  5.93808e-05 | model  |                 5.93808e-05 |        2.78537     |
|      1 | Aborted  |                 15 | warning      |                   0 |            0.00321429 |                 0 |            15.5532  |  2425.67  | caffe       |                       0.001 |          0.00142857  |         52 | 20160704-085910-0cc9 |         40.4331  | 20160704-144437-f9a3 |             15.5532  |                15 | raw AlexFF                           |         6.85933     | 1.46764e+09 |           70.0985  |           6.85683 |          6.85683 |           0.00345238 |                  0.0001      | model  |                 0.0001      |        6.84831     |
|      1 | Done     |                100 | success      |                   0 |            0.499524   |                 0 |            99.9437  | 12800.8   | caffe       |                       0.01  |          0.000833333 |        100 | 20160704-085910-0cc9 |          7.03286 | 20160704-104545-8c3e |             99.9437  |               100 | caffenet_trans_cleanFF_lockConv_new  |         0.141801    | 1.46762e+09 |            7.51231 |           2.72975 |          2.61158 |           0.500595   |                  5.93808e-05 | model  |                 5.93808e-05 |        0.0797321   |
|      1 | Done     |                 30 | success      |                   0 |            0.472957   |                 0 |            29.9321  |  3186.86  | caffe       |                       0.01  |          0.00108173  |        100 | 20160704-085910-0cc9 |          7.23988 | 20160704-090616-ca2e |             29.9321  |                30 | AlexNetTransfer                      |         0.282563    | 1.46762e+09 |            7.67856 |           2.87541 |          2.75557 |           0.475601   |                  1e-05       | model  |                 1e-05       |        0.188101    |

In [53]:
212064000. / (64*375*47*47)

4.0

In [54]:
821694400 / 921600.

891.5954861111111

In [ ]:
# Legacy

total = 0
for k, ps in sorted(param.items()):
    size = reduce(operator.add, [reduce(operator.mul,p) for p in ps])
    total += size      
    print "%12s, %5.1fM, %6.4f GB" % (k, size*10**-6, size * 4 * 2**-30)
    
print "-" * 32
print "%12s, %5.1fM, %6.4f GB" % ("total",total*10**-6, total * 4 * 2**-30)
print "=" * 32